In [1]:
#модули
import math
import random 
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import execute, Aer, circuit, BasicAer
from qiskit.circuit.library import ZGate, RGate, CCXGate, XGate, MCXGate, RC3XGate
import warnings


warnings.filterwarnings('ignore')

# rf_predict

In [2]:
def int_to_bin(int_numb, length):
    s = [0] * length
    i = 0
    while i < length:
        s[i] = int_numb % 2
        int_numb //= 2
        i += 1
    return s 

def mct_ucg(num_control):
    qc = QuantumCircuit(QuantumRegister(num_control + 1))
    gate_list = []
    for _ in range(2 ** num_control - 1):
        gate_list.append([[1, 0], [0, 1]])
    gate_list.append([[0, 1], [1, 0]])
    gate_list = np.array( gate_list)
    qc.uc(list(gate_list), list(range(num_control)), num_control, True)
    return qc.to_gate()

def dec_ucg(num_control=3):
    qc = QuantumCircuit(QuantumRegister(num_control + 1))
    gate_list = []
    for i in range(2 ** 3):
        gate_list.append([[1, 0], [0, 1]])
    gate_list[3] = [[0, 1], [1, 0]] #011 x=0, |help>=1, |control>=1
    gate_list = np.array( gate_list)
    qc.uc(list(gate_list), list(range(num_control)), num_control, True)
    return qc.to_gate()

In [3]:
qc = QuantumCircuit(QuantumRegister(4), ClassicalRegister(4))
qc.x(list(range(4)))
qc.append(mct_ucg(3), list(range(4)))
qc.append(mct_ucg(3).inverse(), list(range(4)))
qc.measure(list(range(4)), list(range(4)))
job = execute(qc, Aer.get_backend('qasm_simulator'), shots=100)
counts = job.result().get_counts(qc)
    
print(counts)

{'1111': 100}


In [4]:
#предсказание на дереве, доводит до листового узла
def tree_predict_controlled(tree, x_len):  
    qc = QuantumCircuit(
        QuantumRegister(1), #|class>
        QuantumRegister(tree.h - 1), #|j>
        QuantumRegister(1), #|help>
        QuantumRegister(x_len), #|x>
        QuantumRegister(1),  #|control>
        QuantumRegister(1)  #|anc_mcx>
    ) 
    qc.cx(1 + tree.h - 1 + 1 + x_len, 1)
    for i in range(1, 2 ** (tree.h - 2)):
        bin_i = int_to_bin(i, tree.h - 2)
        for j in range(len(bin_i)): 
            if bin_i[j] == 0:
                qc.cx(1 + tree.h - 1 + 1 + x_len, j + 1)
        
        if tree.h >= 8:
            qc.mcx(
                list(range(1, tree.h - 1)) + [1 + tree.h - 1 + 1 + x_len],
                [tree.h],
                1 + tree.h - 1 + 1 + x_len + 1, 
                mode='recursion'
            )
        else:
            qc.append(mct_ucg(tree.h - 2 + 1), list(range(1, tree.h - 1)) + [1 + tree.h - 1 + 1 + x_len] + [tree.h])
        
        for j in range(len(bin_i)): 
            if bin_i[j] == 0:
                qc.cx(1 + tree.h - 1 + 1 + x_len, j + 1)
        
        #mult_by_2.controlled()
        for k in range(tree.h - 2, 0, -1):
            #swap(j_k, j_{k-1})
            qc.rcccx(
                [tree.h], [1 + k], [1 + tree.h - 1 + 1 + x_len],
                [1 + k - 1]
                    )
            qc.rcccx(
                [tree.h], [1 + k - 1], [1 + tree.h - 1 + 1 + x_len],
                [1 + k]
                    )
            qc.rcccx(
                [tree.h], [1 + k], [1 + tree.h - 1 + 1 + x_len],
                [1 + k - 1]
                    )
        
        #inc.controlled()
        qc.rcccx(
                [tree.h], [1 + tree.h  + tree.attr_ind[i]], [1 + tree.h - 1 + 1 + x_len],
                [1]
                    )
        
        #inverse |help>
        qc.append(
            mct_ucg(2), [1 + i] + [1 + tree.h - 1 + 1 + x_len] + [1 + tree.h - 1]
        )
    
    for i in range(2 ** (tree.h - 2), 2 ** (tree.h - 1)):
        bin_i = int_to_bin(i, tree.h-1)
        for j in range(len(bin_i)): 
            if bin_i[j] == 0:
                qc.cx(1 + tree.h - 1 + 1 + x_len, j + 1)
        if tree.h >= 8:
            qc.mcx(
                list(range(1, tree.h)) + [1 + tree.h - 1 + 1 + x_len],
                [tree.h], 
                1 + tree.h - 1 + 1 + x_len + 1, 
                mode='recursion'
            )
        else:
            qc.append(mct_ucg(tree.h - 1 + 1), list(range(1, tree.h)) + [1 + tree.h - 1 + 1 + x_len] + [tree.h])
        for j in range(len(bin_i)): 
            if bin_i[j] == 0:
                qc.cx(1 + tree.h - 1 + 1 + x_len, j + 1)
        
        #mult_by_2.controlled()
        for k in range(tree.h - 2, 0, -1):
            #swap(j_k, j_{k-1})
            qc.rcccx(
                [tree.h], [1 + k], [1 + tree.h - 1 + 1 + x_len],
                [1 + k - 1]
                    )
            qc.rcccx(
                [tree.h], [1 + k - 1], [1 + tree.h - 1 + 1 + x_len],
                [1 + k]
                    )
            qc.rcccx(
                [tree.h], [1 + k], [1 + tree.h - 1 + 1 + x_len],
                [1 + k - 1]
                    )
        
        
        #dec.controlled()
        qc.append(
            dec_ucg(), 
            [1 + tree.h - 1 + 1 + x_len] + [tree.h] + [1 + tree.h + tree.attr_ind[i]] + [1]
        ) 
        
        #inverse h
        bin_i = int_to_bin(i * 2, tree.h-1)
        for j in range(1, len(bin_i)): 
            if bin_i[j] == 0:
                qc.cx(1 + tree.h - 1 + 1 + x_len, j + 1)
        if tree.h >= 8:
            qc.mcx(
                list(range(2, tree.h)) + [1 + tree.h - 1 + 1 + x_len],
                [tree.h], 
                1 + tree.h - 1 + 1 + x_len + 1, 
                mode='recursion'
            )
        else:
            qc.append(mct_ucg(tree.h - 2 + 1), list(range(2, tree.h)) + [1 + tree.h - 1 + 1 + x_len] + [tree.h])
        for j in range(1, len(bin_i)): 
            if bin_i[j] == 0:
                qc.cx(1 + tree.h - 1 + 1 + x_len, j + 1)
    
    return qc.to_gate()


In [5]:
def prepare_quantum_x(x):
    qc = QuantumCircuit(QuantumRegister(len(x)))
    for i in range(len(x)):
        if x[i] == '1':
            qc.x(i)
    return qc.to_gate()

In [89]:
class Leaves:
    def __init__(self, leaves_p):
        self.p = leaves_p.copy()
        self.alpha = dict()
        for j in leaves_p:
            self.alpha[j] = math.acos(math.sqrt(leaves_p[j]))

        
class TreeForQuantumPrediction:
    
    
    def __init__(self, h, attr_ind, leaves_proba):
        """Constructor"""
        self.h = h
        self.n_nodes = 2 ** h - 1
        self.attr_ind = attr_ind.copy()
        self.leaves_proba = leaves_proba.copy()
        self.leaves = Leaves(leaves_proba)

        
class RandomForestForQuantumPrediction:

    def __init__(self, h, trees=None): #, trees):
        """Constructor"""
        self.h_trees = h
        if trees is None:
            self.trees = self.__test_trees__() 
        else:
            self.trees = trees.copy()
        self.n_trees = len(self.trees)
        self.angle_list = self.__angels_computing__()

        
    def __angels_computing__(self):
        rf_angle_list = []
        for tree in self.trees:
            for i in range(2 ** (self.h_trees - 1)):
                rf_angle_list.append(2 * tree.leaves.alpha[i])
        return rf_angle_list
    
    
    def __test_trees__(self): 
        tree1 = TreeForQuantumPrediction(3, [-1, 0, 1, 2], {0: 0.02, 1: 0.4, 2: 0.11, 3: 0.65})
        tree2 = TreeForQuantumPrediction(3, [-1, 1, 0, 2], {0: 0.03, 1: 0.75, 2: 0.2, 3: 0.12})
        tree3 = TreeForQuantumPrediction(3, [-1, 2, 0, 1], {0: 0.04, 1: 0.1, 2: 0.6, 3: 0.18})
        tree4 = TreeForQuantumPrediction(3, [-1, 1, 2, 0], {0: 0.07, 1: 0.8, 2: 0.3, 3: 0.03})
        return [tree1, tree2, tree3, tree4]
    
    def predict_proba(self, x):
        p = 0
        for tree in self.trees:
            node = 1
            for _ in range(tree.h - 1):
                if x[tree.attr_ind[node]] == '0':
                    node *= 2
                else:
                    node = node * 2 + 1
            node = node % (2 ** (tree.h - 1))
            p += tree.leaves_proba[node]
        return p / 4
                
            
    
rf = RandomForestForQuantumPrediction(3)        

 

In [90]:
X_test = ['000', '011', '101', '110'] # 4%, 25%, 70%, 11%
for x in X_test:
    print(x, rf.predict_proba(x))

000 0.04
011 0.25
101 0.7
110 0.11000000000000001


In [93]:
for i in range(2 ** 3):
    x = int_to_bin_str(i, 3)
    print(x, rf.predict_proba(x))

000 0.04
001 0.36250000000000004
010 0.23500000000000004
011 0.25
100 0.2575
101 0.7
110 0.11000000000000001
111 0.245


In [7]:
def rf_predict(rf=None, len_x=None):
    len_i = math.ceil(math.log2(rf.n_trees))
    qc = QuantumCircuit(
                            QuantumRegister(1), #|class>
                            QuantumRegister(rf.h_trees - 1), #|j>
                            QuantumRegister(1), #|help>
                            QuantumRegister(len_i), #|i>
                            QuantumRegister(1), #|help>
                            QuantumRegister(len_x), #|X>
                            QuantumRegister(1)  #|anc_mcx> for tree_predict
    )
    qc.h(list(range(1 + rf.h_trees, 1 + rf.h_trees + len_i)))
    for i in range(rf.n_trees):
        bin_i = int_to_bin(i, len_i) 
        for j in range(len(bin_i)):
            if bin_i[j] == 0:
                qc.x(1 + rf.h_trees + j)
        """
        if len_i >= 8:
            qc.mcx(
                list(range(1 + rf.h_trees, 1 + rf.h_trees + len_i)),
                [1 + rf.h_trees + len_i], 
                [1 + rf.h_trees + len_i + 1 + len_x], 
                mode='recursion'
            )
        else:
        """
        qc.append(mct_ucg(len_i), list(range(1 + rf.h_trees, 1 + rf.h_trees + len_i + 1)))
        
       
        qc.append(
            tree_predict_controlled(rf.trees[i], len_x), 
            list(range(1 + rf.h_trees)) 
            + list(range(1 + rf.h_trees + len_i + 1, 1 + rf.h_trees + len_i + 1 + len_x)) 
            + [1 + rf.h_trees + len_i] #|help_rf>
            + [1 + rf.h_trees + len_i + 1 + len_x] #ancilla 
        )
        
        qc.append(mct_ucg(len_i), list(range(1 + rf.h_trees, 1 + rf.h_trees + len_i + 1)))
        """
        qc.mcx(
            list(range(1 + rf.h_trees, 1 + rf.h_trees + len_i)),
            [1 + rf.h_trees + len_i], 
            [1 + rf.h_trees + len_i + 1 + len_x], 
            mode='recursion'
        )
        """
        
        for j in range(len(bin_i)): 
            if bin_i[j] == 0:
                qc.x(1 + rf.h_trees + j)
    qc.ucry(
            rf.angle_list, list(range(1, 1 + rf.h_trees - 1)) + list(range(1 + rf.h_trees, 1 + rf.h_trees + len_i)), 0
        ) 
    return qc.to_gate()
    

In [9]:
def statistics(x, counts):
    k0 = 0
    k1 = 0
    for count in counts:
        if count[-1] == '0':
            k0 += counts[count]
        else:
            k1 += counts[count]
    print(x, 'class_0: ', k0, 'class_1: ', k1)

In [24]:
X_test = ['000', '011', '101', '110'] # 4%, 25%, 70%, 11%
len_i = 2 # the number of trees is 4
len_j = 3 # the trees height is 3
len_X = 3 #the number of attributes 
n = 1 + len_j + len_i + len_X + 1 + 1
counts_test = []
for i in range(len(X_test)):
    qc = QuantumCircuit(
                            QuantumRegister(1), #|class>
                            QuantumRegister(len_j - 1), #|j>
                            QuantumRegister(1), #|help>
                            QuantumRegister(len_i), #|i>
                            QuantumRegister(1), #|help>
                            QuantumRegister(len_X), #|X>
                            QuantumRegister(1), #|anc_mcx> #for tree_predict
                            ClassicalRegister(n)
    )

    qc.append(prepare_quantum_x(X_test[i]), list(range(1 + len_j + len_i + 1, 1 + len_j + len_i + 1 + len_X)))
    qc.append(rf_predict(rf, len_X), list(range(n)))
    qc.measure(list(range(n)), list(range(n)))
    job = execute(qc, Aer.get_backend('qasm_simulator'), shots=100)
    counts = job.result().get_counts(qc)
    statistics(X_test[i], counts)
    print(counts)
    

00000000000 0.00500000000000012
00000000001 0.24500000000000743
00000010000 0.007500000000000229
00000010001 0.24250000000000782
00000100000 0.010000000000000266
00000100001 0.24000000000000835
00000110000 0.01750000000000043
00000110001 0.23250000000000762

000 class_0:  1 class_1:  99
{'00000110001': 21, '00000000001': 16, '00000100001': 35, '00000010001': 27, '00000110000': 1}
01100000010 0.10000000000000109
01100000011 0.15000000000000174
01100010110 0.030000000000000346
01100010111 0.22000000000000225
01100100110 0.0450000000000008
01100100111 0.20500000000000365
01100110100 0.07500000000000212
01100110101 0.1750000000000047

011 class_0:  24 class_1:  76
{'01100010110': 5, '01100110100': 5, '01100100111': 21, '01100000010': 13, '01100100110': 1, '01100010111': 19, '01100110101': 19, '01100000011': 17}
01010000110 0.16250000000000028
01010000111 0.08750000000000045
01010010010 0.18750000000000291
01010010011 0.06250000000000128
01010100100 0.1500000000000045
01010100101 0.10000000

In [42]:
def oracle_for_binary_classification():
    qc = QuantumCircuit(QuantumRegister(1))
    qc.x(0)
    qc.z(0)
    qc.x(0)
    return qc.to_gate()

def s_0(n_qbits):
    qc = QuantumCircuit(QuantumRegister(n_qbits))
    qc.x(list(range(n_qbits)))
    qc.append(ZGate().control(n_qbits - 1), list(range(n_qbits))) #декомпозиция
    qc.x(list(range(n_qbits)))
    return qc.to_gate()

def get_stat(counts):
    k0 = 0
    for count in counts:
        if count[-1] == '0':
            k0 += counts[count]
    return k0

In [10]:
"""
job = execute(qc, BasicAer.get_backend('statevector_simulator'))
vector = job.result().get_statevector()
print_state_vector(vector, n, False)
"""

"\njob = execute(qc, BasicAer.get_backend('statevector_simulator'))\nvector = job.result().get_statevector()\nprint_state_vector(vector, n, False)\n"

In [62]:
def print_state_vector(vector, n, print_proba=True):
    for i in range(len(vector)):
        if abs((vector[i] ** 2).real) >= 0.0001:
            if print_proba:
                print(int_to_bin_str(i, n), abs((vector[i] ** 2)))
            else:
                print(int_to_bin_str(i, n), vector[i])
    print()

    
def int_to_bin_str(a, length):
    s = ''
    while a > 0:
        s = str(a % 2) + s
        a //= 2
    if len(s) < length:
        s = '0' * (length - len(s)) + s
    return s

In [205]:
def get_proba(rf, X, accuracy):
    len_X = len(X)
    len_i = math.ceil(math.log(rf.n_trees, 2))
    len_j = rf.h_trees
    n = 1 + len_j + len_i + len_X + 1 + 1
    for k in range(0, 15):
        qc = QuantumCircuit(
                            QuantumRegister(1), #|class>
                            QuantumRegister(len_j - 1), #|j>
                            QuantumRegister(1), #|help>
                            QuantumRegister(len_i), #|i>
                            QuantumRegister(1), #|help>
                            QuantumRegister(len_X), #|X>
                            QuantumRegister(1), #|anc_mcx> for tree_predict
                            ClassicalRegister(n)
        )
        qc.append(prepare_quantum_x(X_test[i]), list(range(1 + len_j + len_i + 1, 1 + len_j + len_i + 1 + len_X)))
        qc.append(rf_predict(rf, len_X), list(range(n)))

        for _ in range(k):
            #oracle
            qc.append(oracle_for_binary_classification(), [0])
            
            #diffuser
            qc.append(rf_predict(rf, len_X).inverse(), list(range(n)))
            qc.append(s_0(n - len_X - 2), list(range(n -len_X - 2)))
            qc.append(rf_predict(rf, len_X), list(range(n)))
            
        qc.measure(list(range(n)), list(range(n)))
        job = execute(qc, BasicAer.get_backend('qasm_simulator'), shots=100)
        counts = job.result().get_counts(qc)
        k0 = get_stat(counts)
        if k == 0 and k0 / 100 > 0.5:
            return 0, 1
        if k0 / 100 >= accuracy:
            return k, math.sin(math.pi / (4 * k)) ** 2
    return '>15', 0

In [178]:
X_test = [] 
for i in range(2 ** 3):
    x = int_to_bin_str(i, 3)
    X_test.append(x)
    print(x, rf.predict_proba(x))

000 0.04
001 0.36250000000000004
010 0.23500000000000004
011 0.25
100 0.2575
101 0.7
110 0.11000000000000001
111 0.245


In [206]:
rf = RandomForestForQuantumPrediction(3) 
for i in range(len(X_test)):
    k, p = get_proba(rf, X_test[i], 0.85)
    print(X_test[i], 'число запусков QAA:', k, 'вероятность:', p)


000 число запусков QAA: 3 вероятность: 0.06698729810778066
001 число запусков QAA: 1 вероятность: 0.5000000000000001
010 число запусков QAA: 1 вероятность: 0.5000000000000001
011 число запусков QAA: 1 вероятность: 0.5000000000000001
100 число запусков QAA: 1 вероятность: 0.5000000000000001
101 число запусков QAA: 0 вероятность: 1
110 число запусков QAA: 2 вероятность: 0.14644660940672624
111 число запусков QAA: 1 вероятность: 0.5000000000000001


In [176]:
#(math.pi / 4 * 1 / math.asin(math.sqrt(0.11)))

2.3232146820523965

In [181]:
for i in range(len(X_test)):
    qc = QuantumCircuit(
                            QuantumRegister(1), #|class>
                            QuantumRegister(len_j - 1), #|j>
                            QuantumRegister(1), #|help>
                            QuantumRegister(len_i), #|i>
                            QuantumRegister(1), #|help>
                            QuantumRegister(len_X), #|X>
                            QuantumRegister(1), #|anc_mcx> #for tree_predict
                            ClassicalRegister(n)
    )

    qc.append(prepare_quantum_x(X_test[i]), list(range(1 + len_j + len_i + 1, 1 + len_j + len_i + 1 + len_X)))
    qc.append(rf_predict(rf, len_X), list(range(n)))
    
    qc.measure(list(range(n)), list(range(n)))
    #qc.measure(list(range(len_j)) + list(range(len_j + 1, len_j + 1 + len_i)), list(range(len_j + len_i)))
    job = execute(qc, Aer.get_backend('qasm_simulator'), shots=100)
    counts = job.result().get_counts(qc)
    #counts_test.append(counts)
    statistics(X_test[i], counts)
    print(counts)
    

000 class_0:  0 class_1:  100
{'00000100001': 26, '00000000001': 26, '00000110001': 25, '00000010001': 23}
001 class_0:  29 class_1:  71
{'01000010001': 28, '01000000001': 30, '01000100101': 5, '01000110010': 10, '01000110011': 8, '01000100100': 17, '01000010000': 1, '01000000000': 1}
010 class_0:  25 class_1:  75
{'00100010100': 5, '00100100001': 26, '00100000011': 15, '00100010101': 20, '00100110101': 14, '00100110100': 12, '00100000010': 8}
011 class_0:  25 class_1:  75
{'01100000011': 17, '01100000010': 10, '01100110101': 15, '01100100110': 6, '01100010111': 22, '01100100111': 21, '01100110100': 4, '01100010110': 5}
100 class_0:  26 class_1:  74
{'00010100010': 4, '00010010010': 15, '00010000101': 30, '00010010011': 5, '00010100011': 20, '00010110001': 19, '00010000100': 6, '00010110000': 1}
101 class_0:  66 class_1:  34
{'01010100100': 11, '01010000111': 10, '01010010011': 8, '01010110010': 21, '01010000110': 16, '01010100101': 12, '01010110011': 4, '01010010010': 18}
110 class_0: